# This notebook stores each step of refactoring the graph data into PSL data

In [1]:
import pandas as pd
import re
import itertools # for cross products when filling in a full PSL dataset

## These functions help parse the .tab files.

In [2]:
# assigns types to each column
def resolve_column_type(table):
    for column in table.columns:
        if column in {'id', 'email', 'alt_email', 'numsent', 'numreceived', 'numexchanged'}:
            table[column] = table[column].astype(str).astype(float).astype(int)
        # convert bag-of-words columns to floats (since ints won't take NaNs)
        elif re.match("w-", column):
            table[column] = table[column].astype(str).astype(float)

# extracts feature name from an element in a raw tab row
# returns: tuple (feature_name, feature_value, optional_value)
def get_feature_tuple(feature):
    feature_data = re.split(r"[:=]", feature)
    return feature_data
    

# loads the *.tab files into a Pandas Dataframe.
# returns: pd.DataFrame(columns=features)
def load_table(filename):

    # initialize the pandas dataframe
    node_data = pd.DataFrame()


    with open(filename) as infile:
        i = 0
        row_list = []
        for row in infile:
    
            #print('i is: ', i)

            if i == 0:
                # Skip non-useful first line
                print("Header: ", row)
            elif i == 1:
                # Prepare dataframe column labels
                tokens = row.split()
                if len(tokens) == 1:
                    print("This is not a NODE file, so don't load this row")
                else:  
                    features = ["id"] + [get_feature_tuple(feature)[1] for feature in tokens]
                    node_data = pd.DataFrame(columns=features)
            else:
          
                # this is to help the function generalize among the NODE and EDGE files.
                # EDGE files have a "|" character, which needs to be removed for proper feature decoupling
                row = re.sub(r'\|','', row)
            
                tokens = row.split()

                # the first token doesn't need splitting
                row_dict = {'id':tokens[0]}
                row_dict.update({get_feature_tuple(token)[0]:get_feature_tuple(token)[1] for token in tokens[1:]})
                row_list.append(row_dict)
        
            i += 1
        
        # Fill in rows
        node_data = pd.concat([node_data, pd.DataFrame(row_list)], ignore_index=True)

    return node_data

# Process the email nodes

In [3]:
email_nodes = load_table('../c3/namata-kdd11-data/enron/enron-samples-lowunk/enron-sample-lowunk-1of6/sample-enron.NODE.email.tab')
# remove the (unnecessary) second to last column (it came from an ambiguous parse splits)
email_nodes.drop('other,manager,specialist,director,executive', axis=1, inplace=True)
resolve_column_type(email_nodes)

email_nodes.dtypes

Header:  NODE	email



id                int64
emailaddress     object
numsent           int64
numreceived       int64
numexchanged      int64
                 ...   
w-kinney        float64
w-veselack      float64
w-mwhitt        float64
w-jarnold       float64
title            object
Length: 5119, dtype: object

In [4]:
email_nodes

,id,emailaddress,numsent,numreceived,numexchanged,w-gerald,w-know,w-busi,w-mexicana,w-transact,...,w-bartlo,w-columbiagassubject,w-perron,w-coh,w-agl,w-kinney,w-veselack,w-mwhitt,w-jarnold,title
0,98,scott.goodell@enron.com,98,607,705,1.0,1.0,1.0,NaN,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,specialist
1,283,c..koehler@enron.com,128,606,734,1.0,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,director
2,183,p..south@enron.com,8,351,359,1.0,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,director
3,204,lavorato@enron.com,388,3,391,NaN,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,executive
4,303,t..hodge@enron.com,95,570,665,1.0,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,114,vkamins@enron.com,0,12,12,NaN,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,director
207,282,sean.crandall@enron.com,94,138,232,NaN,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,director
208,270,david.duran@enron.com,7,145,152,NaN,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,director
209,243,kevin.presto@enron.com,248,198,446,1.0,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,executive


# Process the CoRef edges

In [5]:
# need to rename one of the columns due to key collision
# use copy for safety
!cp ../c3/namata-kdd11-data/enron/enron-samples-lowunk/enron-sample-lowunk-1of6/sample-enron.UNDIRECTED.coref.tab .
!sed -i 's/email/alt_email/2g' sample-enron.UNDIRECTED.coref.tab

coref_edges = load_table('sample-enron.UNDIRECTED.coref.tab')
resolve_column_type(coref_edges)

coref_edges.dtypes

Header:  UNDIRECTED	coref

This is not a NODE file, so don't load this row


id            int64
email         int64
alt_email     int64
exists       object
dtype: object

In [6]:
coref_edges

,id,email,alt_email,exists
0,2856,265,141,NOTEXIST
1,18491,310,295,NOTEXIST
2,516,272,183,NOTEXIST
3,5131,201,19,NOTEXIST
4,12417,138,78,NOTEXIST
...,...,...,...,...
20776,15003,208,135,NOTEXIST
20777,4450,197,47,NOTEXIST
20778,20302,25,248,NOTEXIST
20779,12985,222,118,NaN


In [7]:
# Sanity Check: These should print pairs of the same people
for index in coref_edges[coref_edges['exists'] == 'EXIST'][['email', 'alt_email']].index:
    email_id  = coref_edges.loc[index]['email']
    alt_email_id = coref_edges.loc[index]['alt_email']

    print(email_nodes[email_nodes['id'] == email_id]['emailaddress'])
    print(email_nodes[email_nodes['id'] == alt_email_id]['emailaddress'])
    print("------------------------------------------------")
    

206    vkamins@enron.com
Name: emailaddress, dtype: object
110    j.kaminski@enron.com
Name: emailaddress, dtype: object
------------------------------------------------
21    d..baughman@enron.com
Name: emailaddress, dtype: object
77    don.baughman@enron.com
Name: emailaddress, dtype: object
------------------------------------------------
209    kevin.presto@enron.com
Name: emailaddress, dtype: object
141    kpresto@enron.com
Name: emailaddress, dtype: object
------------------------------------------------
36    m..tholt@enron.com
Name: emailaddress, dtype: object
43    jane.tholt@enron.com
Name: emailaddress, dtype: object
------------------------------------------------
182    t..lucci@enron.com
Name: emailaddress, dtype: object
16    paul.lucci@enron.com
Name: emailaddress, dtype: object
------------------------------------------------
85    mwhitt@ect.enron.com
Name: emailaddress, dtype: object
136    mark.whitt@enron.com
Name: emailaddress, dtype: object
----------------------

# Process the Manager edges

In [8]:
# Load in the email-submgr and sanity check the edges to see who is the manager of whom.
# need to rename one of the columns due to key collision
# use copy for safety
!cp ../c3/namata-kdd11-data/enron/enron-samples-lowunk/enron-sample-lowunk-1of6/sample-enron.UNDIRECTED.email-submgr.tab .
# FIXME: this is tainting the column names
!sed -i 's/\temail/\talt_email/2g' sample-enron.UNDIRECTED.email-submgr.tab

manager_edges = load_table('sample-enron.UNDIRECTED.email-submgr.tab')
manager_edges.drop('NOTEXIST,EXIST', axis=1, inplace=True)
resolve_column_type(manager_edges)

manager_edges.dtypes

Header:  UNDIRECTED	email-submgr



id                int64
w-gerald        float64
w-know          float64
w-busi          float64
w-mexicana      float64
                 ...   
w-jarnold       float64
numexchanged      int64
email             int64
alt_email         int64
exists           object
Length: 5118, dtype: object

In [9]:
manager_edges

,id,w-gerald,w-know,w-busi,w-mexicana,w-transact,w-want,w-thing,w-review,w-questar,...,w-coh,w-agl,w-kinney,w-veselack,w-mwhitt,w-jarnold,numexchanged,email,alt_email,exists
0,2693,NaN,1.0,NaN,NaN,1.0,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6,286,324,EXIST
1,2634,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,37,74,NOTEXIST
2,1256,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14,148,131,NaN
3,1406,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,313,57,NaN
4,1344,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13,24,170,NOTEXIST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,2105,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13,67,288,NOTEXIST
2047,2374,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,237,198,212,NaN
2048,3464,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,160,210,NOTEXIST
2049,531,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9,316,188,NOTEXIST


# Split data into observed and targets (AKA train and test)

In [10]:
email_nodes_observed = email_nodes[email_nodes['title'].notna()]
email_nodes_targets = email_nodes[email_nodes['title'].isna()]

coref_edges_observed = coref_edges[coref_edges['exists'].notna()]
coref_edges_targets = coref_edges[coref_edges['exists'].isna()]

manager_edges_observed = manager_edges[manager_edges['exists'].notna()]
manager_edges_targets = manager_edges[manager_edges['exists'].isna()]

In [11]:
# Sanity check to see if the splits match up with the paper.

print("email_node_observed: ", len(email_nodes_observed))
print("email_node_target: ", len(email_nodes_targets))

print("coref_edges_observed: ", len(coref_edges_observed))
print("coref_edges_target: ", len(coref_edges_targets))

print("manager_edges_observed: ", len(manager_edges_observed))
print("manager_edges_target: ", len(manager_edges_targets))

email_node_observed:  171
email_node_target:  40
coref_edges_observed:  16625
coref_edges_target:  4156
manager_edges_observed:  1642
manager_edges_target:  409


# Prepare data for PSL predicates

In [12]:
# Takes a table and fills the missing pairs and values to specify a full, sufficient set
# So far it only works with binary predicates
def fill_observed_missing_possibilities(table, arguments, values):
    total_possibilities = set(itertools.product(list(table[arguments[0]]), values))
    already_observed_possibilities = set((table.loc[index][arguments[0]], table.loc[index][arguments[1]]) for index in table.index)

    missing_possibilities = total_possibilities - already_observed_possibilities
    row_list = []
    for arg_0, arg_1 in missing_possibilities:
        row_dict = {arguments[0]:arg_0, arguments[1]:arg_1, arguments[2]:0 }
        row_list.append(row_dict)
        
    return pd.concat([table, pd.DataFrame(row_list)])

## Predicate: EmailHasLabel(E, L)

In [13]:
# convert titles to integers, so PSL can ground faster
title_map = {"other": 0, "manager": 1, "specialist": 2, "director": 3, "executive": 4}

# The copy is to suppress an in-place warning
email_has_label = email_nodes_observed[['id', 'title']].copy()
email_has_label['title'] = email_has_label['title'].map(title_map)

# add in an existence column
email_has_label['exists'] = 1.0

In [14]:
# Specify the full observed set, add in 1s for the observed, and 0s for the missing possibilities
full_set_email_has_label = fill_observed_missing_possibilities(email_has_label, ['id', 'title', 'exists'], list(title_map.values()))
full_set_email_has_label


,id,title,exists
0,98,2,1.0
1,283,3,1.0
2,183,3,1.0
3,204,4,1.0
5,318,4,1.0
...,...,...,...
679,308,0,0.0
680,57,0,0.0
681,202,0,0.0
682,236,2,0.0


In [15]:
# Outputs to file
# full_set_email_has_label.to_csv('EmailHasLabel_obs.csv', sep ='\t', index=False, header=False)

## Predicate: CoRef(E1, E2)

In [16]:
# Outputs to file
# coref_edges.to_csv('CoRef_obs.csv', sep ='\t', index=False, header=False)

## Predicate: EmailManages(E1, E2)